In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
from functions import main, save_df, import_to_sql
from graphics import *

df_all = main("https://www.cheapshark.com/api/1.0/deals")

In [27]:
df_games = df_all[['gameID', 'title', 'release_date', 'scores']].copy()
df_games.drop_duplicates(subset=['gameID'], inplace=True)
df_games = df_games.reset_index(drop=True)
save_df(df_games, "games")

In [28]:
df_shops = df_all[['store_id', 'store_name']].copy()
df_shops.drop_duplicates(subset=['store_id'], inplace=True)
df_shops.reset_index(drop=True)
save_df(df_shops, "shops")

In [29]:
df_gameunits = df_all[['gameID', 'store_id', 'original_price', 'discount_price']].copy()
save_df(df_gameunits, "units")

In [30]:
df_games_genres = df_all[['gameID', 'genre_name']].copy().explode("genre_name")

df_games_genres.insert(0, 'id', range(1, len(df_games_genres) + 1))


save_df(df_games_genres, "games_genres")

In [31]:
df_genres = df_all[['genre_name']].copy().explode('genre_name').drop_duplicates().reset_index(drop=True)
df_genres = df_genres[df_genres['genre_name'].notna()]

df_genres['description'] = 'null'

save_df(df_genres, "genres")

In [32]:

import_to_sql(df_games, 'games')

import_to_sql(df_genres, 'genres')

import_to_sql(df_games_genres, 'games_genres')

import_to_sql(df_shops, 'shops')

import_to_sql(df_gameunits, 'units')


In [33]:
#average_disc_vs_shop_bar(deals_df)
# min_max_disc_vs_shop_bar(deals_df)
# linear_graph_discounts_vs_punct_linear(deals_df)
# genre_vs_mean_discount_bar(unroll_list_from_dfcolumn(deals_df, 'genre'))
# circular_pie_chart_genres(unroll_list_from_dfcolumn(deals_df, 'genre'))
# linear_graph_discounts_vs_releasedate(clean_df_releases(deals_df))

In [ ]:
# streamlit run app.py